<a href="https://colab.research.google.com/github/Aayu0710/Data-Science-ML/blob/main/Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import pandas as pd

#We define some constants to help us later 
CSV_COLUMN_NAME = ['SepalLength', 'SepalWidth', 'PetalLenght', 'PetalWidth','Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

#importing datasets
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")
#read the data from the csv filesn as Panda dataframe
train = pd.read_csv(train_path, names=CSV_COLUMN_NAME, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAME, header=0)

#we usually pop the label or column which we need to predict
y_train = train.pop('Species')
y_test = test.pop('Species')
train.head() #speices column is gone
train.shape # we have 124 entries with 4 features

def input_fun(features, labels, training = True, batch_size=256):
  # now we need to convert the input as dataset object
  # we dont have any epochs as its only classification
  dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
  #shuffling and repaeating
  if training:
    dataset = dataset.shuffle(1000).repeat()
  return dataset.batch(batch_size)


#Feature column
my_feature_column = []
for key in train.keys():
  my_feature_column.append(tf.feature_column.numeric_column(key=key))
print(my_feature_column)

#We will use DDN as n our data may not be any linear depence 
classifier = tf.estimator.DNNClassifier(
    feature_columns = my_feature_column,
    hidden_units =[30,10], # we build a 2 hidden layers of 30 and 10 nodes each
    n_classes = 3) #the model must choose b/w two classes

classifier.train(
    input_fn=lambda: input_fun(train, y_train, training=True),
    steps=5000)

#this is same as x = lamdbda: print("Hi")

#now its time for evalution  of model

eval_result = classifier.evaluate(input_fn=lambda: input_fun(test, y_test, training=False),
    steps=5000)
print('\n test set accuracy: {accuracy:0.3f}\n'.format(**eval_result))


# we need to create an input function again

def input_fn(features, batch_size=256):
  return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)
features =  ['SepalLength', 'SepalWidth', 'PetalLenght', 'PetalWidth']
predict = {}

print("Please enter the features:") 
for feature in features:
  valid = True
  while valid:
    val = input(feature + ": ")
    if not val.isdigit(): valid = False

  predict[feature] = [float(val)]

predictions = classifier.predict(input_fn=lambda:input_fn(predict))
for pred_dict in predictions:
  class_id = pred_dict['class_ids'][0]
  probability = pred_dict['probabilities'][class_id]

  print('prediction is "{}" ({:.1f}%)'.format
  (
      SPECIES[class_id], 100*probability
  ))